In [36]:
instance = instanceId = InstanceId
apiUrl = ApiUrl
schoolYear = SchoolYear
DistrictId = DistrictID = districtId = districtID
apiLimit = batchLimit

prepareSAPMetaData = prepareSAPMetadata
zone = submissionsType = sap_pipeline

In [37]:
from notebookutils import mssparkutils
import configparser

config_path = "/tmp/conf.ini"
def copy_config_to_temp():
    mssparkutils.fs.cp(oea.to_url("stage1/Transactional/SAP/metadata-assets/edfi-configs.ini"),"file:/tmp/conf.ini")

def read_edfi_credentials(config_path):
    config = configparser.ConfigParser()
    config.read(config_path)

    edfi_credentials = {}

    if 'EdFi' in config:
        edfi_credentials['client_id'] = config['EdFi'].get('client_id', '')
        edfi_credentials['client_secret'] = config['EdFi'].get('client_secret', '')

    return edfi_credentials

In [ ]:
%run EdGraph/modules/SAP_PEIMS/v0.6/src/utilities/sap_peim_v0_6_dump_column_mapping

In [38]:
# FIXME: Do not pass uncessary params
input_params = {
        "kvName": kvName,
        "kVName": kvName,
        "workspace": workspace,
        "apiUrl": apiUrl,
        "instanceId": instanceId,
        "moduleName": moduleName,
        "apiLimit": apiLimit,
        "minChangeVer": minChangeVer,
        "maxChangeVer": maxChangeVer,
        "sapVersion": sapVersion,
        "prepareSAPMetaData": prepareSAPMetaData,
        "submissions": submissions,
        "submissionsType": submissionsType,
        "schoolYear": schoolYear,
        "districtId": districtId,
        "districtID": districtId,
        "edfi_url": apiUrl,
        "pipelineExecutionId" : pipelineExecutionId
    }

In [39]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql import Column

from pyspark.sql import functions as F
from pyspark.sql.types import StringType, StructType, StructField

from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
from pyspark.sql.functions import col, struct, concat, lit, round, concat, array, expr, when, date_format, to_date
from pyspark.sql.types import StringType

from pyspark.sql import functions as F
from pyspark.sql.types import StringType, StructType, StructField

from pyspark.sql import SparkSession

import requests
import csv

import json
import uuid
from requests.auth import HTTPBasicAuth
from datetime import datetime
import logging
import json

import threading
import requests
import json
import logging
import queue

from pyspark.sql.types import StructType, StringType
from pyspark.sql import functions as F
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql import functions as F
from pyspark.sql.functions import col


### URLs Initializations

In [40]:
%run OEA/modules/Ed-Fi/v0.7/src/utilities/edfi_v0_7_fetch_urls

In [41]:
instance_id = instanceId
school_year = schoolYear
api_year = school_year
api_url = apiUrl

# FIXME: 2024-01-31 TEMP FIX FOR FY
try:
    edfi_api_manager = EdFiApiManager(api_url, instance_id, api_year)
    edfi_api_manager.update_urls()
    edfi_api_manager.set_other_metadata()

    dependenciesUrl = edfi_api_manager.dependencies_url
    openApiMetadataUrl = edfi_api_manager.openapi_metadata_url
    dataManagementUrl = edfi_api_manager.data_management_url
    authUrl = edfi_api_manager.auth_url

    changeQueriesUrl = edfi_api_manager.get_referenced_url('Change-Queries')
    changeQueriesUrl = changeQueriesUrl[:-13].replace('/metadata/', '/')
    swagger_url = swaggerUrl = edfi_api_manager.get_referenced_url('Resources')

    apiVersion = edfi_api_manager.api_version
    apiVersion = apiVersion[1:] if apiVersion.startswith('v') else apiVersion
except Exception as error:
    edfi_api_manager = EdFiApiManager(api_url, instance_id, '')
    edfi_api_manager.update_urls()
    edfi_api_manager.set_other_metadata()

    dependenciesUrl = edfi_api_manager.dependencies_url
    openApiMetadataUrl = edfi_api_manager.openapi_metadata_url
    dataManagementUrl = edfi_api_manager.data_management_url
    authUrl = edfi_api_manager.auth_url

    changeQueriesUrl = edfi_api_manager.get_referenced_url('Change-Queries')
    changeQueriesUrl = changeQueriesUrl[:-13].replace('/metadata/', '/')
    swagger_url = swaggerUrl = edfi_api_manager.get_referenced_url('Resources')

    apiVersion = edfi_api_manager.api_version
    apiVersion = apiVersion[1:] if apiVersion.startswith('v') else apiVersion

### OEA Initializations

In [42]:
%run EdGraph/modules/SAP_PEIMS/v0.6/src/utilities/sap_peim_v0_6_sap_py

In [43]:
from datetime import datetime
oea = SAPEdFiOEAChild(workspace='dev', 
                      logging_level=logging.INFO, 
                      storage_account=None, 
                      keyvault=None, 
                      timezone=None,
                      sap_pipeline = sap_pipeline,
                      sap_pipelineType = sap_pipelineType)   
oea.set_workspace(workspace)
oea.ingestionHistoryMode=ingestionHistoryMode

In [44]:
# Set Ed-Fi Credentials
# copy_config_to_temp()

# credentials = read_edfi_credentials(config_path)
# client_id = credentials.get('client_id')
# client_secret_id = credentials.get('client_secret')

### Main Code

In [45]:
exception = None
entitiesToFetch = "All"
from datetime import datetime
import math

retry_strategy = Retry(total = 3,
                       backoff_factor = 1,
                       status_forcelist = [429, 500, 502, 503, 504],
                       allowed_methods = ["HEAD", "GET", "OPTIONS", "POST", "DELETE"])
try:
    edfiAPIClient = EdFiClient(workspace = workspace, 
                                    kvName = kvName, #NOTE: Default to None 
                                    moduleName = moduleName, 
                                    authUrl = authUrl, 
                                    dataManagementUrl = dataManagementUrl, 
                                    changeQueriesUrl = changeQueriesUrl, 
                                    dependenciesUrl = dependenciesUrl, 
                                    apiVersion = apiVersion, 
                                    batchLimit = batchLimit, 
                                    minChangeVer = minChangeVer, 
                                    maxChangeVer = maxChangeVer,
                                    landingDateTimeFormat = "yyyyMMddHHmmss",
                                    schoolYear = schoolYear,
                                    districtId = districtId,
                                    kvSecret_clientId = kvSecret_clientId ,
                                    kvSecret_clientSecret = kvSecret_clientSecret,
                                    retry_strategy = retry_strategy,
                                    threadMode = True,
                                    devMode = False)
except Exception as e:
    exception = e
    logger.error(exception)

    raise(exception)

In [51]:
def ingest_descriptor(descriptor):
    output, total_count, failedBatchUrls = edfiAPIClient.returnEntityData(resource = descriptor,
                                            minChangeVersion = None, 
                                            maxChangeVersion = None,
                                            increment = batchLimit // 2)
    if not(output is None or output == []):
        output = spark.createDataFrame(output)
        output = output.dropDuplicates(subset = ['id'])
        if output.count() != total_count:
            logger.error(f'Missing Records - {descriptor} not fetched properly')
            logger.error(f'Actual Num Records - {total_count}')
            logger.error(f'Fetched Num Records - {output.count()}')
    return output

def return_file_path_and_url(file_path):
    column_mapping_file_path = file_path
    column_mapping_file_url = oea.to_url(file_path)

    return column_mapping_file_path, column_mapping_file_url
    
def dump_to_delta(df, 
                  file_path):
    df.write.format('delta').mode('overwrite').save(file_path)

def get_latest_changes(changes_df, sink_path, filtering_date = 'LastModifiedDate',debugMode = False):
        maxdatetime = None
        try:
            sink_df = oea.query(sink_path, f'select max({filtering_date}) maxdatetime')
            maxdatetime = sink_df.first()['maxdatetime']
        except AnalysisException as e:
            pass

        if maxdatetime and not(debugMode):
            changes_df = changes_df.where(f"{filtering_date} > '{maxdatetime}'")        
        return changes_df


In [52]:
entities_info = edfiAPIClient.getEntities()
descriptors = []
for entity_info in entities_info:
    if entity_info['resource'].lower().endswith('descriptors'):
        descriptors.append(entity_info['resource'])

In [53]:
from datetime import datetime
today_date = datetime.utcnow().strftime('%Y%m%d%H%M%S')
def threaded_task(descriptor, base_path):
    descriptor_path = f"{base_path}/{descriptor}"
    descriptor_url = oea.to_url(descriptor_path)
    df = ingest_descriptor(descriptor)
    if df is not None:
        try:
            logger.info(f"[METADATA DESCRIPTOR LANDING THREAD] DESCRIPTOR: {descriptor}")
            df = df.dropDuplicates() # df.dropDuplicates(subset = ['id'])
            df_changes = get_latest_changes(changes_df = df,
                                    sink_path = descriptor_path,
                                    filtering_date = '_lastModifiedDate',
                                    debugMode = False)
            
            if df_changes.count() > 0:
                # df_changes = df_changes.withColumn('RECORD', f.col('id'))
                df_changes = df_changes.withColumn('RECORD', f.concat_ws('_', f.col('namespace'), f.col('codeValue')).cast("String"))

                df_changes = df_changes.withColumn("rundate", F.to_timestamp(F.lit(today_date).cast('string'), landingDateTimeFormat))
                df_changes = df_changes.withColumn('DistrictId', lit(DistrictId))
                df_changes = df_changes.withColumn('SchoolYear', lit(SchoolYear)) 
                df_changes = df_changes.withColumn('sap_pipeline', F.lit(sap_pipeline))
                df_changes = df_changes.withColumn('sap_pipelineType', F.lit(sap_pipelineType))
                df_changes = df_changes.withColumn('stage1_source_url', lit('INVALID_COLUMN_PLACEHOLDER')) # NOTE: placeholder
                # df_changes = df_changes.withColumn('NATURAL_KEY_HASH', f.col('id'))  # NOTE: placeholder 
                df_changes = df_changes.withColumn('NATURAL_KEY_HASH', f.concat_ws('_', f.col('namespace'), f.col('codeValue')).cast("String"))
                # df_changes = df_changes.withColumn('metadata_pipeline_type', lit('NON_SUBMISSION_RECORD'))
                
                dump_to_delta(df = df_changes, 
                              file_path = descriptor_url)
            else:
                logger.info(f'[METADATA DESCRIPTOR LANDING THREAD] Descriptor Table - {descriptor} already has most recent data')
        except Exception as error:
            logger.error(f'[METADATA DESCRIPTOR LANDING THREAD] Unexpected Error - Ingestion of Descriptor - {descriptor} failed')
            logger.error(f'[METADATA DESCRIPTOR LANDING THREAD] Error Info - {error}')
    else:
        logger.info(f'Invalid - {descriptor}')

base_path = f'stage2/Ingested/SAP/descriptorTables/{sapVersion}/DistrictId={districtId}/SchoolYear={schoolYear}'
with ThreadPoolExecutor(max_workers=10) as tpe:
    logger.info('[METADATA DESCRIPTOR LANDING] Entered Threadpool')
    for descriptor in descriptors:
        tpe.submit(threaded_task, descriptor, base_path)    